In [10]:
import torch
import torchvision
import torchvision.models as models
from torchvision.transforms import v2
from torch.utils.data import Dataset, DataLoader
import os
import cv2

In [2]:
from FFTConv import *
from ImageHandler import *

In [3]:
def change_layer(layer):
    fft_conv = FFTConvNet(layer, 'low')
    return fft_conv

In [4]:
model = models.googlenet(weights=None, init_weights=True)
model.aux1 = None
model.aux2 = None

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 1024),
    nn.LeakyReLU(),
    nn.Dropout(p=0.2),
    nn.Linear(1024, 3),
    nn.LogSoftmax(dim=1)
)

for name, module in model.named_modules():
    if name.startswith('inception') and isinstance(module, nn.Conv2d):
        if module.kernel_size[0] > 1:
            fft_conv = change_layer(module)
            parent_name, attr_name = name.rsplit(".", 1)
            parent_module = dict(model.named_modules())[parent_name]
            setattr(parent_module, attr_name, fft_conv)

In [5]:
model.load_state_dict(torch.load(os.path.join('models', 'fft_googlenet_final.pth')))

C:\Users\samir\AppData\Local\Temp\ipykernel_2300\4086650231.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(os.path.join('models', 'fft_

<All keys matched successfully>

In [6]:
image_path = os.path.join('data', 'test_set')
IMG_SIZE = 128
test_data = ImageDataset(image_path)

data\test_set\bacterial


100%|███████████████████████████████████████████████████████████████████████████████| 242/242 [00:01<00:00, 146.54it/s]


data\test_set\normal


100%|████████████████████████████████████████████████████████████████████████████████| 234/234 [00:06<00:00, 38.73it/s]


data\test_set\viral


100%|███████████████████████████████████████████████████████████████████████████████| 148/148 [00:01<00:00, 115.45it/s]


In [11]:
test_dl = DataLoader(test_data, batch_size=24, shuffle=False)

In [8]:
device = 'cuda'
model = model.to(device)

In [9]:
def test(model, test_dl):
    corrects = 0
    total_entries = 0
    model_accuracy = 0
    
    model.eval()
    with torch.no_grad():
        for images, labels in tqdm(test_dl):
            images = images.to(device)
            labels = labels.to(device)
            labels = labels.squeeze().long()
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)

            corrects += torch.sum(predictions == labels).item()
            total_entries += labels.size(0)
        model_accuracy = corrects / total_entries
        print(f'Model Accuracy: {model_accuracy}')
        
    print('Testing Complete')

In [13]:
test(model, test_dl)

100%|██████████████████████████████████████████████████████████████████████████████████| 26/26 [00:25<00:00,  1.03it/s]

Model Accuracy: 0.7884615384615384
Testing Complete
